<a href="https://colab.research.google.com/github/3dsf/ArtLineFFMPEG/blob/main/ArtLineFFMPEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArtLine Video Conversion
implementation of 
https://github.com/vijishmadhavan/ArtLine



# Setup


## Install Libs

In [ ]:
!apt install ffmpeg
!pip install youtube-dl fastai==1.0.61 ffmpeg-python PyDrive

In [ ]:
import os
os.makedirs("input", exist_ok=True)
os.makedirs("output", exist_ok=True)

## Setup Colab to Access Your Google Drive
**Not Required** *, but a mounted drive reduces downloads*
### Copy the shared drive   `'Add to my Drive'` , validate the generated link


*   https://drive.google.com/drive/folders/1cytDlb2ldtw6bq5_o8qZGifRiABruD2a?usp=sharing


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')



## Download Infernce Code

In [ ]:
!wget -O run.py https://gist.githubusercontent.com/3dsf/5827842bd6dc4fe7ada5dda868eb4599/raw/6ffc0b5010e30a9e52af684f752adb2142718961/run.py

# ---> Select your video to process

In [ ]:
youtubeVideo = "https://www.youtube.com/watch?v=pBkHHoOIIn8" #@param {type:"string"}
!rm input.mp4
!time(youtube-dl -f '[ext=mp4]' --output "input.%(ext)s"  $youtubeVideo)

# Decode Video

In [ ]:
#Audio
!rm audio.mp3
!time(ffmpeg -i input.mp4  audio.mp3 -y) 

In [ ]:
#Visual
!rm input/*.bmp output/*.bmp
!time(ffmpeg -i input.mp4 input/frame.%05d.bmp)

# ---> Select the Model to Use
If you **didn't connect** to your **drive** and copy the models, **automatic fallback** is provided. 

In [ ]:
artLineModel = "ArtLine_500.pkl" #@param ["ArtLine_500.pkl", "ArtLine_650.pkl", "ArtLine_1024.pkl"]
pathToModel = os.path.join("/content/drive/MyDrive/ArtLineModels/",artLineModel)

downloadModel = {
    "ArtLine_500.pkl": "https://www.dropbox.com/s/p9lynpwygjmeed2/ArtLine_500.pkl",
    "ArtLine_650.pkl": "https://www.dropbox.com/s/starqc9qd2e1lg1/ArtLine_650.pkl",
    "ArtLine_1024.pkl": "https://www.dropbox.com/s/rq90q9lr9arwdp8/ArtLine_1024%20%281%29.pkl"
}
import os.path as path
import subprocess
if path.isfile(pathToModel) == False : 
    if path.isfile(artLineModel) == False :
        import subprocess
        print("Downloading")
        subprocess.call("wget -O " + artLineModel + " " + downloadModel[artLineModel] , shell=True)
        pathToModel = artLineModel
    else :
        print("Found Local Version")
        pathToModel = artLineModel

# Run Model

In [ ]:
!time(python run.py $pathToModel)

# ---> Select the Output Name and **Encode**


In [ ]:
output_name = "output.webm" #@param {type:"string"}
!time(ffmpeg -framerate `ffmpeg -i input.mp4 2>&1 | sed -n "s/.*, \(.*\) fp.*/\1/p"` -thread_queue_size 8192 -i output/frame.%05d.bmp   -i audio.mp3  $output_name -y)

# Download result

In [ ]:
from google.colab import files

files.download(output_name)